In [1]:
import pickle
import pandas as pd

In [2]:
final_df = pickle.load(open('dataset_level3.pkl','rb'))

In [3]:
X = final_df.drop(columns=['runs_x'])
y = final_df['runs_x']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [4]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
16718,New Zealand,England,Durban,60,60,6,6.000000,31.0
1691,West Indies,Pakistan,Abu Dhabi,67,41,5,5.088608,36.0
12176,England,Pakistan,Manchester,130,23,4,8.041237,32.0
30099,New Zealand,South Africa,Hamilton,117,27,7,7.548387,33.0
30699,Sri Lanka,Pakistan,Abu Dhabi,105,42,7,8.076923,26.0
...,...,...,...,...,...,...,...,...
9762,South Africa,England,Centurion,183,17,6,10.660194,54.0
46264,Sri Lanka,West Indies,Pallekele,127,35,8,8.964706,38.0
18926,Australia,West Indies,Barbados,72,69,7,8.470588,35.0
35423,New Zealand,South Africa,Durban,23,86,7,4.058824,18.0


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [6]:
trf = ColumnTransformer([('trf',
                          OneHotEncoder(sparse=False,drop='first'),
                          ['batting_team','bowling_team','city'])],
                        remainder='passthrough')

In [7]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [8]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.986308450508473
1.7229024536148674


In [9]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [10]:
teams = ['Australia', 'India', 'Bangladesh', 'New Zealand', 'South Africa', 'England', 'West Indies', 'Afghanistan', 'Pakistan', 'Sri Lanka']

cities = ['Colombo', 'Mirpur', 'Johannesburg', 'Dubai', 'Auckland', 'Cape Town', 'London', 'Pallekele', 'Barbados', 'Sydney', 'Melbourne', 
          'Durban', 'St Lucia', 'Wellington', 'Lauderhill', 'Hamilton', 'Centurion', 'Manchester', 'Abu Dhabi', 'Mumbai', 'Nottingham', 
          'Southampton', 'Mount Maunganui', 'Chittagong', 'Kolkata', 'Lahore', 'Delhi', 'Nagpur', 'Chandigarh', 'Adelaide', 'Bangalore', 
          'St Kitts', 'Cardiff', 'Christchurch', 'Trinidad']

# Load the trained model
with open('pipe.pkl', 'rb') as f:
    pipe = pickle.load(f)

def predict_score(batting_team, bowling_team, city, overs, current_score, wickets, last_five):
    balls_left = 120 - (overs * 6)
    wickets_left = 10 - wickets
    crr = current_score / overs

    input_df = pd.DataFrame({
        'batting_team': [batting_team], 
        'bowling_team': [bowling_team], 
        'city': city, 
        'current_score': [current_score],
        'balls_left': [balls_left], 
        'wickets_left': [wickets], 
        'crr': [crr], 
        'last_five': [last_five]
    })
    result = pipe.predict(input_df)
    return int(result[0])

# Sample inputs
batting_team = 'New Zealand'
bowling_team = 'England'
city = 'Christchurch'
overs = 15
current_score = 105
wickets = 4
last_five = 33

# Predict score
predicted_score = predict_score(batting_team, bowling_team, city, overs, current_score, wickets, last_five)
print("Predicted Score:", predicted_score)

Predicted Score: 147


In [11]:
import xgboost as xgb
print(xgb.__version__)

1.4.2
